## 新聞網站爬蟲
### Step1. 最新文章列表

In [1]:
import requests
from bs4 import BeautifulSoup
import re

news_list = []
url = "https://money.udn.com/money/index"

# 1.填寫headers
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
           AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
#print(response.text)

article_title = soup.select('div[class="story__list story__list--web"]')
#print(article_title[:3])

# 2.抓取即時區域所有新聞title以及新聞內容連結link，並將每篇新聞的連結link append到news_list內
for i in article_title[:4]:
    title = i.a["title"].replace(" ","").replace("\n","")
    link = "https://money.udn.com" + i.a["href"]
    print(title)
    print(link + "\n")
    news_list.append(link)

新加坡人愛去台灣玩疫後旅遊新產品熱銷
https://money.udn.com/money/story/5930/6948114?from=edn_newest_index

遠雄集團捐贈400台斤蔬果元宵送暖助等家寶寶
https://money.udn.com/money/story/5621/6948109?from=edn_newest_index

IMF：中國復甦將貢獻全球經濟但有不確定性
https://money.udn.com/money/story/5599/6948104?from=edn_newest_index

美上月新增非農就業51.7萬人遠遠超乎預期
https://money.udn.com/money/story/5599/6948101?from=edn_newest_index



### Step2. 爬取文章內文和圖片並儲存

In [2]:
from time import sleep
import os
from bs4 import BeautifulSoup
from urllib.request import urlretrieve

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

        
def save_news(url):
    # 3.填寫headers
    headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
               AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"} 
    response = requests.get(url, headers=headers)
    sleep(1)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # 4. 抓取每篇新聞的標題和內文
    article_title = soup.select('h1[id="story_art_title"]')
    save_path = "./news/" + article_title[0].text.replace(":",'')
    article_body = soup.select('section[id="article_body"]')
    print("News Title: {}".format(article_title[0].text))
          
    # create folder
    createFolder(save_path)
    soup2 = BeautifulSoup(str(article_body[0]), "html.parser")
    
    # save article contens in folder
    with open(save_path + "/contents.txt", "w", encoding='utf-8') as f:  
        # 5. 抓取文章內p tags，並將新聞內文寫入contents.txt檔案內
        p_tags = soup2.select("p")
        print("News Content: ")
        for i in p_tags:
            print(i.text)
            f.write(i.text)
          
    # save image in article        
    # 6. 抓取文章內所有img tags，並將圖片下載並存入對應新聞資料夾下
    img_tags = soup2.select("img")
    print("Save Image: ")
    for i in img_tags:
        print(i["src"])
        urlretrieve(i["src"], save_path+"/"+i["title"]+".jpg")
    print("==============================\n")

          
for news_url in news_list:
    print(news_url)
    save_news(news_url)

https://money.udn.com/money/story/5930/6948114?from=edn_newest_index
News Title: 新加坡人愛去台灣玩 疫後旅遊新產品熱銷
News Content: 
台灣開放國境後，外國旅客赴台人數逐步攀升。新加坡人向來愛到台灣旅遊，業者表示，疫後深度旅遊需求增加，主打台灣燈會、廟會文化等產品熱賣，原本的百人團暴增至200多人。

台灣去年10月開放國境，據觀光局統計，去年11月、12月新加坡赴台旅客人次為5萬9250。星國旅遊業者卓瓊華日前接受中央社記者訪問時表示，預期今年旅客回流狀況會更好，已有不少客人計劃要去台灣看櫻花、梧桐花。

「台灣對新加坡人有莫名的吸引力」，她說，台星兩地航程近、沒有時差，台灣有四季，有溫泉、老街、農場、大自然景觀，也有現代的一面，吸引不同年齡層的新加坡旅客。

卓瓊華本身也非常喜歡到台灣旅遊，她形容，「台灣幾乎是我第2個家」。疫情前每年都會去3、4次，一定要泡溫泉、在台北逛逛吃吃，還要在休閒農場過夜。

隨著全球疫情趨緩，跨境旅遊逐步復甦，她指出，疫情改變許多人對旅遊的想法。以前可能是「點到為止」，每天換一個城市，但疫情過後，深度旅遊的需求增加，旅客想在同一地點待久一點、慢慢玩，停留3天2夜，也比較偏好郊區的行程。

特別的是，也有當地旅行社配合5日將在台北登場的台灣燈會推出「燈會團」，由從台灣到新加坡發展的歌台藝人「皓皓」帶團，為期一週的行程結合台灣文化與美食。

卓瓊華表示，這個特別行程團預計6日出發，行程涵蓋台北、彰化、苗栗等地，遊客有機會親自體驗台灣本土過年及廟會文化。

儘管2月並非新加坡人出國旅遊的旺季，但這個行程一推出就「被秒殺」，她說，原本只規劃100人，「沒想到2天內就賣光」，後來應客人要求，整團名額增至250多人。看到台灣開放後，旅客反應這麼好，「我們是滿開心的」。

除了燈會團，卓瓊華表示，疫後也依台灣不同區域規劃深度旅遊行程，例如花東之旅、南部之旅等產品。

有些人喜歡跟團，有些旅客則偏好自由行。觀光局駐新加坡辦事處主任鄭智鴻受訪表示，到辦事處詢問旅遊資訊的新加坡遊客多以自由行為主，但與以往不同的是，疫情之後，詢問自駕、台灣新景點的遊客人數變多了。

他預估，今年整體新加坡赴台旅客人次有望恢復至疫情前的50%，也就是23萬人次左右。

據觀光局統